## Homework

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

## Install packages

In [ ]:
!pip install pandas
!pip install pyarrow
!pip install scikit-learn

## Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2022.

In [4]:
!curl -o taxi_trips_january.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet
!curl -o taxi_trips_february.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 37 36.3M   37 13.4M    0     0  10.6M      0  0:00:03  0:00:01  0:00:02 10.6M
100 36.3M  100 36.3M    0     0  20.2M      0  0:00:01  0:00:01 --:--:-- 20.2M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 43.5M    0  110k    0     0   288k      0  0:02:34 --:--:--  0:02:34  288k
 33 43.5M   33 14.6M    0     0  10.6M      0  0:00:04  0:00:01  0:00:03 10.6M
100 43.5M  100 43.5M    0     0  18.3M      0  0:00:02  0:00:02 --:--:-- 18.3M


## Reading data

In [51]:
!python -V

Python 3.11.2


In [78]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression


In [98]:

trips_january = pd.read_parquet('taxi_trips_january.parquet')
trips_january = pd.read_parquet('taxi_trips_february.parquet')


In [99]:
trips_january.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-02-01 00:06:58,2022-02-01 00:19:24,1.0,5.40,1.0,N,138,252,1,17.0,1.75,0.5,3.90,0.00,0.3,23.45,0.0,1.25
1,1,2022-02-01 00:38:22,2022-02-01 00:55:55,1.0,6.40,1.0,N,138,41,2,21.0,1.75,0.5,0.00,6.55,0.3,30.10,0.0,1.25
2,1,2022-02-01 00:03:20,2022-02-01 00:26:59,1.0,12.50,1.0,N,138,200,2,35.5,1.75,0.5,0.00,6.55,0.3,44.60,0.0,1.25
3,2,2022-02-01 00:08:00,2022-02-01 00:28:05,1.0,9.88,1.0,N,239,200,2,28.0,0.50,0.5,0.00,3.00,0.3,34.80,2.5,0.00
4,2,2022-02-01 00:06:48,2022-02-01 00:33:07,1.0,12.16,1.0,N,138,125,1,35.5,0.50,0.5,8.11,0.00,0.3,48.66,2.5,1.25


## Q1. Downloading the data

Read the data for January. How many columns are there?



In [94]:
print(f'There are {len(trips_january.columns)} columns in data for january')

There are 19 columns in data for january


## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

In [100]:
trips_january['duration'] = trips_january.tpep_dropoff_datetime - trips_january.tpep_pickup_datetime
trips_january['duration'] =  trips_january.duration.apply(lambda td: td.total_seconds() / 60)

In [96]:
std_trips_duration =  trips_january['duration'].std()
print(f'The standard deviation of trips in January is {std_trips_duration}')

The standard deviation of trips in January is 47.26394076778834


## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [101]:
len_original_data = len(trips_january)
trips_january = trips_january[(trips_january.duration >= 1) & (trips_january.duration <= 60)]
print(len(trips_january)/len_original_data)

0.979444397269143


The fraction of records left after drop outliers is 98%

## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [103]:
categorical = ['PULocationID', 'DOLocationID']
trips_january[categorical] = trips_january[categorical].astype(str)

In [104]:
train_dicts = trips_january[categorical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
len(dv.feature_names_)

514

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters 
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [105]:
from sklearn.metrics import mean_squared_error

target = 'duration'
y_train = trips_january[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

7.639501432962458

In [83]:
mean_squared_error(y_train, y_pred, squared=True)

2184.490669841498

In [82]:
46.73853**0.5

6.836558344664367